In [3]:
"""
Step: Tile Chipping and Mask Generation

This step divides a large georeferenced satellite raster (TIFF) and its polygon labels (shapefile) into smaller, fixed-size tiles for machine learning model training.

- Cleans up previous tile outputs to avoid mixing results from different runs.
- Slides a fixed window over the raster to extract image chips and records their coordinates.
- Skips "void" (mostly black) tiles to save disk space, but still logs their location for later mask stitching.
- For each valid tile:
    - Saves the RGB chip as a PNG image.
    - Rasterizes intersecting polygons from the shapefile to generate a per-pixel class mask (with class ID remapping), saving as PNG.
- Writes out metadata for every tile, including skipped tiles, to a CSV file for traceability and post-processing.
- Stores the original raster shape for later use (e.g., reconstructing full-size predictions).

Inputs:
    - GeoTIFF raster image
    - Polygon shapefile with 'class_id' field

Outputs:
    - Image tiles (PNG)
    - Mask tiles (PNG, class IDs remapped to YOLO format, 255=ignore)
    - Metadata CSV (tile name, x, y)
    - Raster shape file

Purpose:
    - Prepares georeferenced training data for semantic segmentation models (e.g., YOLOv8-seg), ensuring spatial alignment between image chips and polygon masks.
"""

import os
import csv
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
import cv2
from tqdm import tqdm
import shutil

# --- Config ---
map_folder = "Flight_2"
base_dir = "C:/QGIS"
chip_size = 1024
stride = 922

# Original → YOLO class IDs
class_remap = {
    1: 0,  # Road
    2: 1,  # PVeg
    3: 2   # Water
}

map_base = os.path.join(base_dir, map_folder)
tif_path = os.path.join(map_base, f"{map_folder}.tiff")
shp_path = os.path.join(map_base, f"{map_folder}.shp")

out_img_dir = os.path.join(map_base, "tiled", "images")
out_mask_dir = os.path.join(map_base, "tiled", "masks")
meta_csv = os.path.join(map_base, "tiled", "tile_metadata.csv")
shape_txt = os.path.join(map_base, "tiled", "raster_shape.txt")

# --- Clean old data ---
for path in [out_img_dir, out_mask_dir]:
    if os.path.exists(path):
        shutil.rmtree(path)
os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_mask_dir, exist_ok=True)

if os.path.exists(meta_csv):
    os.remove(meta_csv)
if os.path.exists(shape_txt):
    os.remove(shape_txt)

# --- Open raster and store shape ---
with rasterio.open(tif_path) as raster:
    raster_height, raster_width = raster.height, raster.width
    raster_crs = raster.crs
    with open(shape_txt, "w") as f:
        f.write(f"{raster_height},{raster_width}")

# --- Load and reproject shapefile ---
labels = gpd.read_file(shp_path).to_crs(raster_crs)
sindex = labels.sindex

# --- Begin processing ---
results = []
skipped = 0

with rasterio.open(tif_path) as raster:
    for idx, (y, x) in enumerate(tqdm(
        [(y, x) for y in range(0, raster_height - chip_size + 1, stride)
                 for x in range(0, raster_width - chip_size + 1, stride)],
        desc="🌍 Chipping tiles"
    )):
        window = Window(x, y, chip_size, chip_size)
        transform = raster.window_transform(window)
        bounds = box(*rasterio.windows.bounds(window, raster.transform))

        # Read image chip
        img = raster.read([1, 2, 3], window=window)
        img = np.transpose(img, (1, 2, 0))
        img_name = f"chip_{idx}.png"

        # --- Skip writing black tiles, but keep metadata ---
        if np.mean(img) < 5 and np.std(img) < 2:
            skipped += 1
            results.append((img_name, x, y))
            continue

        # Save image
        cv2.imwrite(os.path.join(out_img_dir, img_name), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

        # Rasterize mask
        possible_matches = labels.iloc[list(sindex.intersection(bounds.bounds))]
        intersecting = possible_matches[possible_matches.intersects(bounds)]
        if not intersecting.empty:
            shapes = [
                (geom, class_remap.get(cid, 255))
                for geom, cid in zip(intersecting.geometry, intersecting["class_id"])
            ]
            mask = rasterize(
                shapes,
                out_shape=(chip_size, chip_size),
                transform=transform,
                fill=255,
                dtype=np.uint8
            )
        else:
            mask = np.full((chip_size, chip_size), 255, dtype=np.uint8)

        cv2.imwrite(os.path.join(out_mask_dir, img_name), mask)
        results.append((img_name, x, y))

# --- Save metadata ---
with open(meta_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "x", "y"])
    writer.writerows(results)

print(f"✅ {len(results)} tiles processed.")
print(f"🧹 {skipped} black/void tiles skipped (not saved, but tracked for stitching).")

🌍 Chipping tiles: 100%|███████████████████████████████████████████████████████████| 3519/3519 [06:13<00:00,  9.41it/s]


✅ 3519 tiles processed.
🧹 148 black/void tiles skipped (not saved, but tracked for stitching).


In [4]:
"""
Step: Convert Tiled Images and Masks to YOLO Polygon Format

This step processes the image and mask tiles generated in the previous step to prepare them for YOLOv8-seg training.  
For each dataset/map folder:

- Resizes all image and mask tiles to a fixed target size (e.g., 640×640).
- Converts each mask into YOLO-style polygon label files:
    - Finds all external contours (polygons) for each class present in the mask (ignoring 255/void).
    - Normalizes polygon coordinates to [0, 1] range as required by YOLO format.
    - Writes a `.txt` label file for each tile, where each line represents one polygon as:  
      `<class_id> x1 y1 x2 y2 ... xn yn`
- Images and corresponding labels are saved into the appropriate train/val subdirectories, based on a user-defined validation set.
- Uses multithreading for faster processing of large datasets.
- Logs statistics for each map: number of labeled tiles, background-only tiles (no features), and errors.

**Inputs:**
    - Image tiles (PNG)
    - Mask tiles (PNG, grayscale class mask)

**Outputs:**
    - YOLO-ready images (resized PNGs)
    - YOLO polygon label files (`.txt`, one per image)
    - Directory structure matching YOLOv8 expectations:
        - images/train, labels/train
        - images/val, labels/val

**Purpose:**
    - Converts georeferenced segmentation data into the format required for YOLOv8 polygon segmentation training, enabling efficient, batch model development across multiple datasets/maps.

"""

import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Configuration ---
base_dir = "C:/QGIS"
target_size = 640
num_threads = 8

all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]

validation_set = "Flight_2"

def mask_to_polygons(mask):
    contours = {}
    for cls_id in np.unique(mask):
        if cls_id == 255:
            continue
        binary = (mask == cls_id).astype(np.uint8)
        cnts, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            contours[cls_id] = cnts
    return contours

def process_file(fname, img_input_dir, mask_input_dir, img_out, lbl_out):
    img_path = os.path.join(img_input_dir, fname)
    mask_path = os.path.join(mask_input_dir, fname)

    try:
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or mask is None:
            return "error"

        img_resized = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)
        mask_resized = cv2.resize(mask, (target_size, target_size), interpolation=cv2.INTER_NEAREST)

        label_path = os.path.join(lbl_out, fname.replace(".png", ".txt"))
        contours = mask_to_polygons(mask_resized)

        # Save image
        cv2.imwrite(os.path.join(img_out, fname), img_resized)

        # Write label file
        with open(label_path, "w") as f:
            if not contours:
                return "background"
            for cls_id, cnts in contours.items():
                for cnt in cnts:
                    if len(cnt) < 3:
                        continue
                    pts = cnt.reshape(-1, 2).astype(np.float32) / target_size
                    coords = " ".join(f"{x:.6f} {y:.6f}" for x, y in pts)
                    f.write(f"{cls_id} {coords}\n")

        return "labeled"
    except Exception as e:
        print(f"⚠️ Error processing {fname}: {e}")
        return "error"

# --- Process each map ---
for map_folder in all_maps:
    print(f"\n🧩 Converting: {map_folder}")

    base_map_dir = os.path.join(base_dir, map_folder)
    img_input_dir = os.path.join(base_map_dir, "tiled", "images")
    mask_input_dir = os.path.join(base_map_dir, "tiled", "masks")
    out_base = os.path.join(base_map_dir, "yolo_dataset_640")

    # Create output directories
    if map_folder == validation_set:
        img_out = os.path.join(out_base, "images", "val")
        lbl_out = os.path.join(out_base, "labels", "val")
    else:
        img_out = os.path.join(out_base, "images", "train")
        lbl_out = os.path.join(out_base, "labels", "train")
    
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    # Process all files
    counts = {"labeled": 0, "background": 0, "error": 0}
    chip_files = [f for f in os.listdir(img_input_dir) if f.endswith(".png")]

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_file, f, img_input_dir, mask_input_dir, img_out, lbl_out) for f in chip_files]
        for f in tqdm(as_completed(futures), total=len(futures), desc=f"{map_folder}"):
            result = f.result()
            counts[result] += 1

    print(f"✅ {map_folder} summary:")
    print(f"   - {counts['labeled']} labeled tiles")
    print(f"   - {counts['background']} background-only tiles")
    print(f"   - {counts['error']} errors")


🧩 Converting: Bear_Creek_20250112


Bear_Creek_20250112: 100%|█████████████████████████████████████████████████████████| 2874/2874 [00:32<00:00, 88.62it/s]


✅ Bear_Creek_20250112 summary:
   - 2494 labeled tiles
   - 380 background-only tiles
   - 0 errors

🧩 Converting: Bear_Lane


Bear_Lane: 100%|███████████████████████████████████████████████████████████████████| 1020/1020 [00:16<00:00, 62.30it/s]


✅ Bear_Lane summary:
   - 953 labeled tiles
   - 67 background-only tiles
   - 0 errors

🧩 Converting: Flight_2


...

✅ Sugar_Refugia_20241112 summary:
   - 959 labeled tiles
   - 170 background-only tiles
   - 0 errors

🧩 Converting: Wildcat_Creek


Wildcat_Creek: 100%|███████████████████████████████████████████████████████████████| 1195/1195 [00:25<00:00, 47.03it/s]


✅ Wildcat_Creek summary:
   - 699 labeled tiles
   - 496 background-only tiles
   - 0 errors

🧩 Converting: Project_2024_09_20


Project_2024_09_20: 100%|██████████████████████████████████████████████████████████| 2061/2061 [00:29<00:00, 71.01it/s]

✅ Project_2024_09_20 summary:
   - 1666 labeled tiles
   - 395 background-only tiles
   - 0 errors


In [1]:
"""
Step: Merge and Prefix YOLO Datasets for Training

This step consolidates preprocessed YOLO-format datasets from multiple map sources
into a single unified dataset directory for model training and validation.

- Creates the required directory structure (`images/train`, `images/val`, `labels/train`, `labels/val`) in a master dataset folder.
- Cleans out any old data to avoid mixing results from previous runs.
- For each map set:
    - Skips any explicitly excluded sets (e.g., partial or downsampled variants).
    - Locates YOLO-format train/val images and labels generated by the previous pipeline step.
    - Copies images and labels into the master dataset, prefixing filenames with their map set name for traceability.
    - Ensures the designated validation set (e.g., "Flight_2") is copied into `val` directories, with all others going into `train`.
    - Creates empty label files for images missing a label, ensuring each image has a corresponding label file as required by YOLO.

**Inputs:**
    - Multiple folders with YOLO-style image and label files, already split into train/val subsets.

**Outputs:**
    - Unified dataset directory with images and labels for both training and validation, ready for YOLOv8-seg training.

**Purpose:**
    - Combines all map-specific datasets into a single training/validation set with unique, traceable file naming,
      simplifying model training and evaluation on a multi-map corpus.

"""

import os
import shutil
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"
dataset_dir = os.path.join(base_dir, "dataset")
img_train_dir = os.path.join(dataset_dir, "images", "train")
img_val_dir = os.path.join(dataset_dir, "images", "val")
lbl_train_dir = os.path.join(dataset_dir, "labels", "train")
lbl_val_dir = os.path.join(dataset_dir, "labels", "val")

validation_set = "Flight_2"
exclude_set = "Flight_2_25pct"

all_sets = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]

# --- Create necessary directories ---
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)

# --- Clean old contents ---
for folder in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    for f in os.listdir(folder):
        file_path = os.path.join(folder, f)
        if os.path.isfile(file_path):
            os.remove(file_path)

# --- Copy pre-sorted train/val files from each set ---
for set_name in tqdm(all_sets, desc="Copying datasets"):
    # Skip the excluded set
    if set_name == exclude_set:
        print(f"⏭️ Skipping excluded set: {exclude_set}")
        continue

    base_map_dir = os.path.join(base_dir, set_name, "yolo_dataset_640")
    for mode in ["train", "val"]:
        img_src_dir = os.path.join(base_map_dir, "images", mode)
        lbl_src_dir = os.path.join(base_map_dir, "labels", mode)

        if not os.path.exists(img_src_dir) or not os.path.exists(lbl_src_dir):
            continue

        # Determine output directories
        if set_name == validation_set:
            img_dst_dir = img_val_dir
            lbl_dst_dir = lbl_val_dir
        else:
            img_dst_dir = img_train_dir
            lbl_dst_dir = lbl_train_dir

        # Copy images and labels with prefix
        for img_file in os.listdir(img_src_dir):
            if img_file.endswith(".png"):
                prefix = set_name + "_"
                img_dst = os.path.join(img_dst_dir, prefix + img_file)
                lbl_dst = os.path.join(lbl_dst_dir, prefix + img_file.replace(".png", ".txt"))

                # Copy image
                shutil.copy(os.path.join(img_src_dir, img_file), img_dst)

                # Copy label
                lbl_src = os.path.join(lbl_src_dir, img_file.replace(".png", ".txt"))
                if os.path.exists(lbl_src):
                    shutil.copy(lbl_src, lbl_dst)
                else:
                    open(lbl_dst, "w").close()

print(f"✅ Merged datasets into: {dataset_dir}")

Copying datasets: 100%|██████████████████████████████████████████████████████████████████| 7/7 [03:19<00:00, 28.49s/it]

✅ Merged datasets into: C:/QGIS\dataset


In [ ]:
# --- Training done in Train.py 
import os
from ultralytics import YOLO
import torch

def main():
    torch.cuda.empty_cache()

    dataset_dir = "C:/QGIS/dataset"
    model_type = "yolov8s-seg.pt"
    img_size = 1024
    save_dir = "C:/QGIS/runs/segment/train"
    class_names = ["Water", "Road", "PVeg"]

    data_yaml = os.path.join(dataset_dir, "data.yaml")
    with open(data_yaml, "w") as f:
        f.write(f"path: {dataset_dir}\n")
        f.write("train: images/train\n")
        f.write("val: images/val\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")

    print(f"✅ data.yaml written to: {data_yaml}")

    model = YOLO(model_type)
    model.train(
        data=data_yaml,
        imgsz=img_size,
        epochs=50,
        batch=16,
        workers=2,
        amp=True,
        patience=5,
        device="cuda",
        save=True,
        save_period=-1,
        project=save_dir,
        name="main_model",
        verbose=True,
        plots=False,
        cache=True
    )

if __name__ == "__main__":
    main()


In [ ]:
# Model Report

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# --- Paths ---
results_path = r"C:/QGIS/runs/segment/train/main_model/results.csv"
report_path = "C:/QGIS/diagnostics/model_report.pdf"
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# --- Load results and clean columns ---
df = pd.read_csv(results_path)
df.columns = df.columns.str.strip()
11
# --- Key epoch metrics ---
final_epoch = df.iloc[-1]
best_map_epoch = df["metrics/mAP50(B)"].idxmax()
best_row = df.iloc[best_map_epoch]

# --- Basic training config (update if needed) ---
training_config = {
    "Model": "YOLOv8n-seg",
    "Input Size": "640x640",
    "Epochs": len(df),
    "Batch Size": 8,
    "Optimizer": "SGD (default)",
    "Confidence Threshold": 0.1,
    "Tile Size": "1024x1024",
    "Mask Source": "Polygon label → Raster mask via cv2.fillPoly"
}

# --- Metrics to plot ---
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5–0.95",
    "metrics/precision(B)": "Precision",
    "metrics/recall(B)": "Recall"
}
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

# --- Create PDF report ---
with PdfPages(report_path) as pdf:
    # Title + Summary page
    plt.figure(figsize=(11, 8.5))
    plt.text(0.5, 0.78, "Model Performance Report", ha="center", fontsize=24)
    plt.text(0.5, 0.70, "Project: River and Road Segmentation", ha="center", fontsize=14)
    plt.text(0.5, 0.63, f"Best mAP@0.5: {best_row['metrics/mAP50(B)']:.3f} at epoch {best_map_epoch}", ha="center", fontsize=12)
    plt.text(0.5, 0.57, f"Final Epoch Precision: {final_epoch['metrics/precision(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.51, f"Final Epoch Recall: {final_epoch['metrics/recall(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.45, "Report generated from YOLOv8 results.csv", ha="center", fontsize=10)
    plt.axis("off")
    pdf.savefig()
    plt.close()

    # Training configuration page
    fig, ax = plt.subplots(figsize=(11, 4))
    ax.axis("off")
    table_data = list(training_config.items())
    table = ax.table(cellText=table_data, colLabels=["Parameter", "Value"], loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.4)
    plt.title("Training Configuration", fontsize=14)
    pdf.savefig()
    plt.close()

    # Metric plots
    for i, (key, label) in enumerate(metrics.items()):
        if key in df.columns:
            plt.figure(figsize=(10, 4))
            plt.plot(df[key], marker='o', linewidth=2, color=colors[i])
            plt.title(f"{label} Over Epochs")
            plt.xlabel("Epoch")
            plt.ylabel(label)
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    # Performance summary table
    summary_data = {
        "Metric": ["mAP@0.5", "mAP@0.5–0.95", "Precision", "Recall"],
        "Final Epoch": [
            f"{final_epoch.get('metrics/mAP50(B)', 0):.3f}",
            f"{final_epoch.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{final_epoch.get('metrics/precision(B)', 0):.3f}",
            f"{final_epoch.get('metrics/recall(B)', 0):.3f}"
        ],
        f"Best Epoch ({best_map_epoch})": [
            f"{best_row.get('metrics/mAP50(B)', 0):.3f}",
            f"{best_row.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{best_row.get('metrics/precision(B)', 0):.3f}",
            f"{best_row.get('metrics/recall(B)', 0):.3f}"
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    fig, ax = plt.subplots(figsize=(10, 2.5))
    ax.axis("off")
    table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    plt.title("Summary of Model Performance", fontsize=14)
    pdf.savefig()
    plt.close()

print(f"✅ PDF report saved to: {report_path}")

✅ PDF report saved to: C:/QGIS/diagnostics/model_report.pdf


In [3]:
"""
Step: Run YOLOv8-Seg Inference and Export Polygon Labels

This step uses a trained YOLOv8-segmentation model to generate predictions on a set of tiled images.

- Loads the trained YOLO model from disk.
- Scans the specified directory of test images (chips) for inference.
- Optionally cleans out old prediction label files to avoid confusion or mixing results.
- Runs prediction in streaming mode for efficiency and stability, using a low confidence threshold (e.g., 0.1) to maximize recall.
- Saves predicted polygons in YOLO format (`.txt`), with one file per image, suitable for downstream mask stitching or evaluation.
- Outputs are saved to a dedicated folder within the map directory, using a consistent naming scheme for traceability.

**Inputs:**
    - Trained YOLOv8-segmentation model (`.pt` checkpoint)
    - Directory of image tiles to predict on

**Outputs:**
    - Polygon label files (`.txt`), YOLO polygon format, one per image
    - All results organized in a `predictions/predict_txt/labels` subdirectory

**Purpose:**
    - Applies a trained model to new/unlabeled image tiles, generating segmentation predictions as polygons for downstream tasks (e.g., mask reconstruction, quantitative evaluation, or visualization).

"""

from ultralytics import YOLO
import torch
import os
import glob

# --- Config ---
map_folder = "Flight_2_25pct"
image_dir = rf"C:\QGIS\{map_folder}\tiled\images"

# Absolute path to trained model
model_path = r"C:\QGIS\runs\segment\train\main_model\weights\best.pt"

# Output folder for predictions
output_dir = rf"C:\QGIS\{map_folder}\predictions"
output_name = "predict_txt"

# --- Load model directly ---
model = YOLO(model_path)

# Output folder for predictions
output_dir = rf"C:\QGIS\{map_folder}\predictions"
output_name = "predict_txt"

# --- Optional: Clean up old predictions ---
label_dir = os.path.join(output_dir, output_name, "labels")
if os.path.exists(label_dir):
    old_labels = glob.glob(os.path.join(label_dir, "*.txt"))
    for f in old_labels:
        os.remove(f)
    print(f"🧹 Cleared {len(old_labels)} old prediction files from: {label_dir}")

# --- Predict with streaming ---
results = model.predict(
    source=image_dir,
    imgsz=640,
    conf=0.1,
    save=False,
    save_txt=True,
    save_conf=False,
    retina_masks=True,
    exist_ok=True,
    project=output_dir,
    name=output_name,
    device="cuda" if torch.cuda.is_available() else "cpu",
    stream=True
)

# Force the generator to run
for _ in results:
    pass

print(f"✅ Inference complete. Labels saved to: {output_dir}\\{output_name}\\labels\\")


image 1/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_103.png: 640x640 4 PVegs, 23.5ms
image 2/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_104.png: 640x640 2 Roads, 6 PVegs, 1 Water, 26.2ms
image 3/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_105.png: 640x640 1 Road, 7 PVegs, 25.5ms
image 4/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_106.png: 640x640 9 PVegs, 25.2ms
image 5/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_107.png: 640x640 15 PVegs, 22.1ms
image 6/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_108.png: 640x640 12 PVegs, 21.7ms
image 7/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_109.png: 640x640 21 PVegs, 26.7ms
image 8/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_110.png: 640x640 23 PVegs, 22.1ms
image 9/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_111.png: 640x640 29 PVegs, 22.9ms
image 10/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_112.png: 640x640 14 PVegs, 30.1ms
image 11/253 C:\QGIS\Flight_2_25pct\tiled\images\chip_113.png: 640x640 11 PVegs, 22.3ms
image 12/253 C:\QG

In [4]:
"""
Step: Stitch YOLO Polygon Predictions into Full-Map Shapefile

This step aggregates polygon predictions from YOLOv8 inference (per-tile `.txt` label files)
and reconstructs them into a single georeferenced shapefile for the entire map.

- Loads chip/tile metadata (filenames, offsets) and the original raster’s georeferencing info.
- Iterates over every predicted label file:
    - Parses YOLO polygon label format for each tile, skipping degenerate/invalid polygons.
    - Rescales polygon coordinates from YOLO-inference resolution to original chip size, then shifts them to the correct map position using tile offsets.
    - Converts tile-relative coordinates to full raster geocoordinates using the original raster’s affine transform.
    - Optionally remaps YOLO class IDs back to original dataset class IDs for compatibility.
    - Collects all valid polygons with associated class IDs.
- Assembles all polygons into a single GeoDataFrame with the original CRS.
- Writes out the complete set of predictions as a georeferenced shapefile, suitable for direct GIS analysis, visualization, or further post-processing.

**Inputs:**
    - YOLO polygon label files (per-tile `.txt`)
    - Tile metadata CSV (for spatial offsets)
    - Raster TIFF (for georeferencing)
    - Class ID mapping (optional, for consistency with ground truth)

**Outputs:**
    - Georeferenced polygon shapefile, with one row per predicted polygon and a class ID column

**Purpose:**
    - Reconstructs a seamless, georeferenced map of predicted features (roads, streams, etc.)
      from individually inferred tiles, closing the loop from deep learning output back to standard GIS data formats.

"""

import os
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from tqdm import tqdm

# --- Config ---
map_folder = "Flight_2_25pct"
base_dir = f"C:/QGIS/{map_folder}"
mask_dir = os.path.join(base_dir, "predictions", "predict_txt", "labels")
tile_metadata_path = os.path.join(base_dir, "tiled", "tile_metadata.csv")
raster_shape_path = os.path.join(base_dir, "tiled", "raster_shape.txt")
tif_path = os.path.join(base_dir, f"{map_folder}.tiff")
shapefile_path = os.path.join(base_dir, f"{map_folder} Segmentation.shp")

chip_size = 1024        # Original chip size used during tiling
inference_size = 640    # YOLOv8 inference resolution

# Optional: Remap YOLO class IDs → Original class IDs
reverse_remap = {
    0: 1,  # Road
    1: 2,  # PVeg
    2: 3   # Water
}

# --- Load metadata and raster georeferencing ---
tile_meta = pd.read_csv(tile_metadata_path)
with open(raster_shape_path, "r") as f:
    height, width = map(int, f.read().strip().split(","))

with rasterio.open(tif_path) as src:
    transform = src.transform
    crs = src.crs

# --- Collect all polygons from predicted labels ---
features = []

for _, row in tqdm(tile_meta.iterrows(), total=len(tile_meta), desc="Stitching Tiles"):
    fname, x, y = row["filename"], int(row["x"]), int(row["y"])
    label_path = os.path.join(mask_dir, fname.replace(".png", ".txt"))
    if not os.path.exists(label_path):
        continue

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:
            continue  # Skip degenerate polygons

        cls_id = int(float(parts[0]))  # Already YOLO-aligned: 0 = Road, 1 = PVeg, 2 = Water

        coords = list(map(float, parts[1:]))
        pts = np.array(coords, dtype=np.float32).reshape(-1, 2)
        pts *= chip_size  # Rescale to original resolution

        pts[:, 0] += x
        pts[:, 1] += y

        geo_pts = [rasterio.transform.xy(transform, y_, x_, offset='center') for x_, y_ in pts]
        poly = Polygon(geo_pts)

        if poly.is_valid and poly.area > 0:
            features.append({
                "geometry": poly,
                "class_id": reverse_remap.get(cls_id, cls_id)  # Optional remap
            })

# --- Export to shapefile ---
gdf = gpd.GeoDataFrame(features, crs=crs)
gdf.to_file(shapefile_path)

print(f"✅ Shapefile saved to: {shapefile_path}")

Stitching Tiles: 100%|███████████████████████████████████████████████████████████████| 667/667 [01:40<00:00,  6.66it/s]


✅ Shapefile saved to: C:/QGIS/Flight_2_25pct\Flight_2_25pct Segmentation.shp


In [6]:
"""
Step: Summarize Predicted Area Coverage by Class

This step calculates the total ground area (in square feet) covered by each predicted class label
(e.g., Road, PVeg, Water) based on YOLO polygon label outputs, and exports the summary to a CSV.

- Reads the ground sample distance (GSD) from the original GeoTIFF to determine area per pixel.
- Iterates through all tile label files:
    - Parses polygons and class IDs from YOLO-format prediction files.
    - Counts the total number of (x, y) coordinate pairs for each class, as a proxy for the number of pixels or points labeled per class.
- Converts the raw pixel/point count to physical area using GSD and appropriate unit conversion (meters to square feet).
- Writes a CSV summary with the total area for each class, ready for client reporting or further analysis.

**Inputs:**
    - GeoTIFF file (for GSD/area calculation)
    - YOLO polygon label files (per-tile `.txt`)
    - Tile metadata CSV

**Outputs:**
    - CSV file summarizing total area (sqft) for each predicted class

**Purpose:**
    - Provides an at-a-glance summary of model-predicted coverage for key features (roads, water, etc.) across the entire map, suitable for quantitative analysis or reporting.

"""

import os
import cv2
import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"
map_folder = "Flight_2_25pct"

reverse_remap = {
    0: "Road",
    1: "PVeg",
    2: "Water"
}

# --- Calculate areas for each map ---
print(f"Calculating areas for: {map_folder}")
base_map_dir = os.path.join(base_dir, map_folder)
tif_path = os.path.join(base_map_dir, f"{map_folder}.tiff")
mask_dir = os.path.join(base_map_dir, "predictions", "predict_txt", "labels")
tile_metadata_path = os.path.join(base_map_dir, "tiled", "tile_metadata.csv")
area_csv_path = os.path.join(base_map_dir, f"{map_folder}_area_summary.csv")

# --- Extract GSD from GeoTIFF ---
with rasterio.open(tif_path) as src:
    gsd_x = abs(src.transform.a)  # Meters per pixel (x direction)
    gsd_y = abs(src.transform.e)  # Meters per pixel (y direction)
    sqft_per_pixel = (gsd_x * gsd_y) * 10.7639  # Correct area calculation

print(f"🌎 {map_folder} GSD: {gsd_x:.4f} x {gsd_y:.4f} meters per pixel ({sqft_per_pixel:.4f} sqft per pixel)")

# --- Collect pixel counts per class ---
class_areas = {"Road": 0, "PVeg": 0, "Water": 0}
tile_meta = pd.read_csv(tile_metadata_path)
for _, row in tile_meta.iterrows():
    fname, x, y = row["filename"], int(row["x"]), int(row["y"])
    label_path = os.path.join(mask_dir, fname.replace(".png", ".txt"))
    if not os.path.exists(label_path):
        continue

    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 7:
                continue  # Skip degenerate polygons

            cls_id = int(float(parts[0]))
            original_class = reverse_remap.get(cls_id, cls_id)
            pixel_count = (len(parts) - 1) // 2  # Each (x, y) pair is 2 coordinates

            # Accumulate area
            class_areas[original_class] += pixel_count

# --- Convert to square feet and round ---
class_areas_sqft = {
    cls_name: round(count * sqft_per_pixel)
    for cls_name, count in class_areas.items()
}

# --- Save CSV for client ---
if os.path.exists(area_csv_path):
    os.remove(area_csv_path)

pd.DataFrame([
    {"label_name": cls_name, "area_sqft": area}
    for cls_name, area in class_areas_sqft.items()
]).to_csv(area_csv_path, index=False)

print(f"✅ {map_folder} area CSV saved to: {area_csv_path}")

Calculating areas for: Flight_2_25pct
🌎 Flight_2_25pct GSD: 0.1712 x 0.1712 meters per pixel (0.3156 sqft per pixel)
✅ Flight_2_25pct area CSV saved to: C:/QGIS\Flight_2_25pct\Flight_2_25pct_area_summary.csv
